In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import pandas as pd

In [ ]:
rows_interval = 8
rows_min = rows_interval
rows_max = 4096
# rows_max = 4294967296
number_of_rows = int(rows_max/rows_interval) - int(rows_min/rows_interval) + 1
row_space = list(range(rows_min, rows_max + 1, rows_interval))

ns_conversion_factor = 1e-6 # ns to ms

In [ ]:
def array_operation(rows, prec):
    try:
        directoryname = f"../blas_results/axpy_{prec}/{rows}"
        filename = "results.csv"
        df = pd.read_csv(f"{directoryname}/{filename}", usecols=["BeginNs", "EndNs"])
        subtracted_df = df["EndNs"] - df["BeginNs"]
        subtracted_array = subtracted_df.to_numpy() * ns_conversion_factor
        return subtracted_array
    except:
        print(rows, prec)

In [ ]:
colors = ['grey', 'purple', 'yellow', 'purple', 'grey']

precisions = [32, 64]

fig, axes = plt.subplots(
    2,
    1,
    dpi=512,
    subplot_kw=dict(box_aspect=1/4),
    sharex=True,
)
fig.tight_layout()
plt.subplots_adjust(hspace=-0.4)

for precision, ax in zip([64, 32], axes):    
    quantile_arrays = [np.zeros(len(row_space)) for _ in range(5)]
    for i, row in enumerate(row_space):
        for j, quantile in enumerate([0,0.25,0.5,0.75,1.0]):
            quantile_arrays[j][i] = np.quantile(array_operation(row, precision), quantile)
    for k, quantile_array in enumerate(quantile_arrays[:3]):
        c = ax.fill_between(
            row_space,
            quantile_array,
            quantile_arrays[4-k],
            color=colors[k],
            zorder=3
        )
    ax.set_ylabel(fr"FP{precision}" + '\n' + "Time to execute (ms)")
    ticks_scalar = 64
    if precision == 32:
        ax.set_xticks(range(ticks_scalar*rows_min, rows_max+1, ticks_scalar*rows_interval))
        ax.set_xlabel("Number of rows ($n$)")
    else:
        ax.xaxis.set_ticks_position('none')
    ylims = 0.004, 0.008
    ax.set_ylim(ylims[0], ylims[1])
    for i in [256, 512, 1024, 2048]:
        ax.vlines(i, ylims[0], ylims[1], color='white', linewidth=0.75, zorder=0)
    print(f"{precision}: 248, 256, 264")
    print(quantile_arrays[2][30:33])
    print(f"{precision}: 504, 512, 520")
    print(quantile_arrays[2][62:65])
    print(f"{precision}: 1016, 1024, 1032")
    print(quantile_arrays[2][126:129])

fig.suptitle(
    r"median runtime for `hipblasXaxpy` kernel"+'\n'+f"for arrays with $n$ rows"
)

plt.show()

In [ ]:
def events_array_operation(rows, prec):
    try:
        directoryname = f"../blas_results/axpy_{prec}/{rows}"
        filename = "events.csv"
        df = pd.read_csv(f"{directoryname}/{filename}", usecols=["L2CacheHit"])
        return df.to_numpy()
    except:
        print(rows, prec)

In [ ]:
colors = ['grey', 'purple', 'yellow', 'purple', 'grey']

precisions = [32, 64]

fig, axes = plt.subplots(
    2,
    1,
    dpi=512,
    subplot_kw=dict(box_aspect=1/4),
    sharex=True,
)
fig.tight_layout()
plt.subplots_adjust(hspace=-0.4)

for precision, ax in zip([64, 32], axes):    
    quantile_arrays = [np.zeros(len(row_space)) for _ in range(5)]
    for i, row in enumerate(row_space):
        for j, quantile in enumerate([0,0.25,0.5,0.75,1.0]):
            quantile_arrays[j][i] = np.quantile(events_array_operation(row, precision), quantile)
    for k, quantile_array in enumerate(quantile_arrays[:3]):
        c = ax.fill_between(
            row_space,
            quantile_array,
            quantile_arrays[4-k],
            color=colors[k],
            zorder=3
        )
    ax.set_ylabel(fr"FP{precision}" + '\n' + "L2 Cache Hit (%)")
    ticks_scalar = 64
    if precision == 32:
        ax.set_xticks(range(ticks_scalar*rows_min, rows_max+1, ticks_scalar*rows_interval))
        ax.set_xlabel("Number of rows ($n$)")
    else:
        ax.xaxis.set_ticks_position('none') 
    ylims = 0, 100
    ax.set_ylim(ylims[0], ylims[1])
    for i in [256, 512, 1024, 2048]:
        ax.vlines(i, ylims[0], ylims[1], color='white', linewidth=0.75, zorder=1)

fig.suptitle(
    r"median L2 Cache Hit for `hipblasXaxpy` kernel"+'\n'+f"for arrays with $n$ rows"
)

plt.show()